In [1]:
import sys
sys.path.insert(0, "/users/jwindmil/2019_WMI/util")

# Initial imports
import Landau_Potential_Diffusion as Landau
import curvature as curve

import xarray as xr
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
import datetime
from skimage import measure
from scipy import ndimage, stats
import scipy.integrate as sin
from scipy.optimize import curve_fit
from scipy.signal import argrelextrema
from scipy.ndimage.filters import gaussian_filter1d
import random
import pandas as pd
import dask.array as da
import matplotlib as mpl

import pickle

In [2]:
def edgTObin(edges):
    return 0.5*(edges[1:]+edges[:-1])

In [3]:
def cm_to_inch(m_cm):
    return m_cm/2.54

In [4]:
def find_index(time_A, time_cond):
    ind = np.where(time_A == time_cond)[0][0]
    return ind 

In [5]:
def get_all_values(dic_values):
    values = []
    for i_k, key in enumerate(dic_values.keys()):
        values.extend(dic_values[key])
    return values

# Potential and LFE 

i) RCE

ii) ERA 5

In [7]:
dic_Fig2 = {'V':{}, 'H':{}, 'CL':{}, 'PDF':{}}

### RCE

Load RCE data

In [8]:
path_RCE = '/project/s916/davidle/RCE-MIP/simulations/RCE_300_3km_506x506/output/'

In [9]:
RCE300 = xr.open_mfdataset(path_RCE+'lfff????????.nc', chunks={'time':100}, combine='by_coords')

Calculate CWV tendencies

In [10]:
dt = 3600; # Timestep in seconds
tcoor = dt*np.arange(0,RCE300.time.shape[0])

t_range = np.arange(0,np.size(RCE300.time)-2)
dPW_dt = (RCE300.TQV.values[t_range+2,:,:]-RCE300.TQV.values[t_range,:,:])/(2*dt)
dPW_dt = np.concatenate((dPW_dt[0:1,:,:],dPW_dt,np.tile(dPW_dt[-1,:,:],(1,1,1))),axis=0)

Calculate Potential from CWV tendencies

In [11]:
bin0=np.percentile(a=RCE300.TQV[-24*7:,:,:],q=50,axis=(0,1,2))
tmp,binm,V = Landau.Landau_energy(RCE300.TQV,dPW_dt,bin0,N_bins=30)

In [12]:
dbin = (binm[1:]-binm[:-1])[0]
binc = binm-0.5*dbin
binc = np.append(binc, binc[-1]+dbin)

In [13]:
dic_Fig2['V']['RCE']={'V':V, 'bin_edge':binm}

Calculate Hamiltonian from CWV field and fixed potential

In [14]:
Ffinal_RCE,binm_Ffinal_RCE,tmp = Landau.Landau_energy(RCE300.TQV.values,N_bins=30,V_fixed=V,bin_fixed=binm)

In [15]:
dic_Fig2['H']['RCE']={'H':Ffinal_RCE, 'time':tcoor}

Calculate Histogram

In [16]:
SPINstart = 4 #day
SPINend   = 6 #day

COALstart = 9 #day
COALend   = 11#day

COARstart =79 #day
ENDstart  =81 #day

In [17]:
CWV_PDF_SPIN,SPINedges = np.histogram(RCE300.TQV[(24*SPINstart):(24*SPINend),:,:],bins=binc,density=True)
CWV_PDF_COAL,COALedges = np.histogram(RCE300.TQV[(24*COALstart):(24*COALend),:,:],bins=binc,density=True)
CWV_PDF_COAR,COARedges = np.histogram(RCE300.TQV[(24*COARstart):(24*ENDstart),:,:],bins=binc,density=True)

In [18]:
dic_Fig2['PDF']['RCE']={'1':[CWV_PDF_SPIN,SPINedges], '2':[CWV_PDF_COAL,COALedges], '3':[CWV_PDF_COAR,COARedges], 'time':[5,10,80]}

Calculate Contour

In [19]:
dx = 3.3e3
CL = np.zeros((np.size(tcoor),))

perc_thresh_RCE = 88

for it,t in enumerate(tcoor):
    print('it=',it,'           ',end='\r')
    CWV_tmp = RCE300.TQV[it,:,:]
    
    CWV_binary = np.zeros(np.shape(CWV_tmp))
    CWV_binary[CWV_tmp>np.percentile(RCE300.TQV[it,:,:], perc_thresh_RCE)] = 1

    binary_boundary=np.copy(CWV_binary)
    binary_boundary[:,1:-1]=0

    L = dx*(measure.perimeter(CWV_binary,8)- np.sum(binary_boundary))
        
    CL[it] = L

In [20]:
dic_Fig2['CL']['RCE']={'CL':CL, 'time':tcoor}

### ERA 5

In [22]:
path_ERA = '/project/s916/ERA5_Tom/'

In [23]:
thresh_era5 = 83

Restrict our analysis to a band of the tropical Atlantic

In [24]:
latmin = -23
latmax = 23
lonmin = 360-34
lonmax = 360-18

In [25]:
date1, date2 = np.datetime64('2000-01-01T00:00'), np.datetime64('2017-12-31T23:00')
dt = 3600

In [26]:
PW = xr.open_mfdataset(path_ERA+'????/??PW.nc',combine='by_coords', chunks={'time':100})

In [27]:
PWAtl = PW['tcwv'].sel({'time':slice(date1, date2), 'longitude':slice(lonmin,lonmax),'latitude':slice(latmax,latmin)})
lonAtl = PW.longitude.sel({'longitude':slice(lonmin,lonmax)})
latAtl = PW.latitude.sel({'latitude':slice(latmax,latmin)})

In [28]:
time_A = PWAtl.time.values

In [29]:
CONJUL_data = {}

path_PKL = '/users/jwindmil/2019_WMI/dev/jwindmiller/PKL_DATA/'

for i,year in enumerate(range(2000,2018)):
    print('i=',i,' & year=',year,' ',end='\r')
    hf = open(path_PKL+'CONTOURL_PW_%i_%i_%i_%i_%i_'%(latmin, latmax, lonmin, lonmax, thresh_era5)+str(year)+'.pkl','rb') # open('../jwindmiller/PKL_DATA/10_17_CONTOURL'+str(year)+'.pkl','rb')
    tmp = pickle.load(hf)
    CONJUL_data[year] = tmp['Tot_Contour_km'][str(year)]
    
cont_t = np.array(get_all_values(CONJUL_data))

In [32]:
with open('../jwindmiller/PKL_DATA/pik_times.dat', 'rb') as f:
    times_A = pickle.load(f)

[numpy.datetime64('2005-11-23T14:00:00.000000000'),
 numpy.datetime64('2005-12-04T14:00:00.000000000'),
 numpy.datetime64('2005-12-07T14:00:00.000000000')]

In [48]:
time_period = time_A[np.where((time_A>times_A[0]-np.timedelta64(6,'h'))&(time_A<times_A[-1]+np.timedelta64(6,'h')))]
cont_period = cont_t[np.where((time_A>times_A[0]-np.timedelta64(6,'h'))&(time_A<times_A[-1]+np.timedelta64(6,'h')))]

In [49]:
dic_Fig2['CL']['ERA5']={'CL':cont_period, 'time':time_period}

#### Calculate Potential and LFE

In [33]:
times_Vstart = times_A[0] 
times_Vend   = times_A[1] 

ileftV = find_index(time_A, np.datetime64(times_Vstart))
irightV = find_index(time_A, np.datetime64(times_Vend))

In [34]:
PWAtl_res = PWAtl[ileftV-1:irightV+2,:,:].values # If can't fit in the memory, can't calculate potential

In [35]:
dPW_dt = (PWAtl_res[2:,:,:]-PWAtl_res[:-2,:,:])/(2*dt)

Calculate Potential

In [36]:
tmp,binm_Vfinal,Vfinal = Landau.Landau_energy(PWAtl_res[1:-1,:,:],dPW_dt,N_bins=50)

Calculate Potential (dis-aggregation phase)

In [37]:
times_Vstart_DA = times_A[1] 
times_Vend_DA   = times_A[2] 

ileftV_DA = find_index(time_A, np.datetime64(times_Vstart_DA))
irightV_DA = find_index(time_A, np.datetime64(times_Vend_DA))

PWAtl_res_DA = PWAtl[ileftV_DA-1:irightV_DA+2,:,:].values # If can't fit in the memory, can't calculate potential

dPW_dt_DA = (PWAtl_res_DA[2:,:,:]-PWAtl_res_DA[:-2,:,:])/(2*dt)

tmp_DA,binm_Vfinal_DA,Vfinal_DA = Landau.Landau_energy(PWAtl_res_DA[1:-1,:,:],dPW_dt_DA,N_bins=50)

In [51]:
dic_Fig2['V']['ERA5']={'Agg':{'V':Vfinal, 'bin_edge':binm_Vfinal}, 'Dis':{'V':Vfinal_DA, 'bin_edge':binm_Vfinal_DA}}

Calculate LFE

In [39]:
date_startF = times_A[0] 
date_endF   = times_A[-1]

ileftF = find_index(time_A, np.datetime64(date_startF))
irightF = find_index(time_A, np.datetime64(date_endF))
PWAtl_F = PWAtl[ileftF:irightF,:,:]
Ffinal,binm_Ffinal,tmp = Landau.Landau_energy(PWAtl_F.values,N_bins=30,V_fixed=Vfinal,bin_fixed=binm_Vfinal)

In [56]:
dic_Fig2['H']['ERA5']={'H':Ffinal, 'time':time_A[ileftF:irightF]}

Calculate histograms of CWV using bins of Potential

In [41]:
dbin_Vfinal = (binm_Vfinal[1:]-binm_Vfinal[:-1])[0]
binc_Vfinal = binm_Vfinal-0.5*dbin_Vfinal
binc_Vfinal = np.append(binc_Vfinal, binc_Vfinal[-1]+dbin_Vfinal)

In [42]:
dic_hist = {}
times_labels = pd.to_datetime(times_A).strftime("%d-%b")
times_year   = int(pd.to_datetime(times_A[0]).strftime("%Y"))

for i, time in enumerate(times_A):
    tmp = np.squeeze(PWAtl.sel({'time':slice(time-np.timedelta64(6,'h'), time+np.timedelta64(6,'h'))}).values)
    hist, edges = np.histogram(np.ndarray.flatten(tmp), density = True, bins = binc_Vfinal)
    
    dic_hist[times_labels[i]]=hist
    
dic_hist.keys()

dict_keys(['23-Nov', '04-Dec', '07-Dec'])

In [43]:
dic_Fig2['PDF']['ERA5']={'1':[dic_hist['23-Nov'], edges], '2':[dic_hist['04-Dec'], edges], '3':[dic_hist['07-Dec'], edges],
                        'time':['23-Nov', '04-Dec', '07-Dec']}

In [57]:
with open('./PKL_DATA/dic_Fig2.pickle', 'wb') as handle:
    pickle.dump(dic_Fig2, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('./PKL_DATA/dic_Fig2.pickle', 'rb') as handle:
#    b = pickle.load(handle)